In [41]:
def parse(s):
    ret="{\n"
    i1=s.find('=',0)
    i2=s.find('&',0)
    prev_v=0
    while(i1!=-1 and i2!=-1):
        s1=s[prev_v:i1]
        s2=s[i1+1:i2]
        prev_v=i2+1
        ret+="\t"+s1+": '"+s2+"',\n"
        i1=s.find('=',i2+1)
        i2=s.find('&',i2+1)
    if(i1!=-1):
        s1=s[prev_v:i1]
        s2=s[i1+1:]
        ret+="\t"+s1+": '"+s2+"',\n"
    ret=ret[:-2]
    ret+='\n}'
    return ret

In [42]:
print(parse("foo=bar&baz=qux&zap=zazzle"))
print(parse("email=foo@bar.com&uid=10&role=user"))

{
	foo: 'bar',
	baz: 'qux',
	zap: 'zazzle'
}
{
	email: 'foo@bar.com',
	uid: '10',
	role: 'user'
}


In [54]:
def profile_for(email):
    email=email.replace("&","")
    email=email.replace("=","")
    return parse("email="+email+"&uid=10&role=user")
print(profile_for("foo@bar.com"))

{
	email: 'foo@bar.com',
	uid: '10',
	role: 'user'
}


In [55]:
import random
from Cryptodome.Cipher import AES
def generate_random_bytes(no_of_bytes):
    return bytearray([random.randint(0,255) for i in range(no_of_bytes)])
block_size=16
key=bytes(generate_random_bytes(block_size))
def do_pad(s):
    reqd=block_size-len(s)%block_size
    if(len(s)<block_size):
        reqd=block_size-len(s)
    if(reqd==0):
        reqd=block_size
    s+=bytearray([4]*reqd)
    return s
def encryption_oracle(s):
    s=do_pad(s)
    #     print(s," ",len(s))
    cipher=AES.new(key,AES.MODE_ECB)
    return bytearray(cipher.encrypt(bytes(s)))
def decryption_oracle(s):
    cipher=AES.new(key,AES.MODE_ECB)
    return cipher.decrypt(bytes(s))

In [56]:
text=profile_for("foo@bar.com")
print(text)
enc=encryption_oracle(bytearray(text,'ISO-8859-1'))
print(enc)
dec=decryption_oracle(enc)
print(dec.decode())

{
	email: 'foo@bar.com',
	uid: '10',
	role: 'user'
}
bytearray(b'\x91\x0c^\xdc\xc4B\x00\x95\x82\x87\xd2;\x87\xba\xf8\x180\xc4\xdf.\x84fm\xc9\x99\x13k\xbfa\x03\x07\xae\xc6\xc1\xee|=\x00\x83jp5\x87\xa6hN\x1d\x8b\xd0(k\x1a\xf2Ub\xfeP\xc9>\x14\x8d?\x7f\xe7')
{
	email: 'foo@bar.com',
	uid: '10',
	role: 'user'
}


In [70]:
str1="email=@gmail.com&uid=10&role="
len1=len(str1)
# print(len1)
len2=3*block_size-len1
while(len2<=0):
    len2+=block_size
str2='S'*len2+"@gmail.com"
# print(str2)
str3="email="+str2+"&uid=10&role="
print(str3)
# print(len(str3))

email=SSSSSSSSSSSSSSSSSSS@gmail.com&uid=10&role=


In [71]:
text1=b''
enc1=encryption_oracle(text1)
print(enc1)
text2=bytearray(str3,'ISO-8859-1')
enc2=encryption_oracle(text2)
print(enc2)

bytearray(b'\x82\xb5\xab\x05\xb8`F\xb1\x97(\xbbd\x0b\x81C\xdf')
bytearray(b'\x11K\x90\x9c\xb5\xcc\x16S\x90\xa7"\xf3Vv4F\xf8hagf\x7f&:H\xe0* F\xd5\xd0\xb5\xd9!q\xed\xd5\xb8\xc3\xa38}\xc6\xf7\x0b\xe8\xc5U\x82\xb5\xab\x05\xb8`F\xb1\x97(\xbbd\x0b\x81C\xdf')


In [76]:
first_part_cipher=enc2[:48]
print(first_part_cipher)
print(decryption_oracle(first_part_cipher))

bytearray(b'\x11K\x90\x9c\xb5\xcc\x16S\x90\xa7"\xf3Vv4F\xf8hagf\x7f&:H\xe0* F\xd5\xd0\xb5\xd9!q\xed\xd5\xb8\xc3\xa38}\xc6\xf7\x0b\xe8\xc5U')
b'email=SSSSSSSSSSSSSSSSSSS@gmail.com&uid=10&role='


In [81]:
text3=b'admin'
second_part_cipher=encryption_oracle(text3)
print(second_part_cipher)

bytearray(b'\x9d[n[<\x1f\xb2\x9f/}=\x04\xae\xc8t\x85')


In [97]:
cipher_text=first_part_cipher+second_part_cipher
print(cipher_text)
plain_text=decryption_oracle(cipher_text)
print(plain_text.decode())
print(parse(plain_text.decode()))

bytearray(b'\x11K\x90\x9c\xb5\xcc\x16S\x90\xa7"\xf3Vv4F\xf8hagf\x7f&:H\xe0* F\xd5\xd0\xb5\xd9!q\xed\xd5\xb8\xc3\xa38}\xc6\xf7\x0b\xe8\xc5U\x9d[n[<\x1f\xb2\x9f/}=\x04\xae\xc8t\x85')
email=SSSSSSSSSSSSSSSSSSS@gmail.com&uid=10&role=admin
{
	email: 'SSSSSSSSSSSSSSSSSSS@gmail.com',
	uid: '10',
	role: 'admin'
}
